Source of data: https://www.kaggle.com/stoney71/new-york-city-transport-statistics

Four CSV files totalling 5GB in size. 

Due to size of data, files will be loaded on Google Drive and not Github.

Each data set contained a last column with no header. Due to this, a header name was provided "None" to bypass errors loading data into a dataframe.

Facts about data

This data shows the first 5 business days of the months June 2017, August 2017, October 2017, and December 2017, with 10 minute increments, of all NYC public transit.

I need to split DF for Buses and trains.

#**Build Environment and EDA**

In [ ]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import datetime as dt

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree, ensemble
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report 

import matplotlib.pyplot as plt

#**CSV1**

In [ ]:
# Load each CSV from local repository to Notebook
# then merge all into one dataframe
Date1 = pd.read_csv("fullmta_1706.csv")
Date2 = pd.read_csv("fullmta_1708.csv")
Date3 = pd.read_csv("fullmta_1710.csv")
Date4 = pd.read_csv("fullmta_1712.csv")

Datesfull = pd.concat([Date1,Date2,Date3,Date4])

In [ ]:
# Reset index on merged file, 
# we here then see over 4 million rows of data
Datesfull.reset_index(drop=True, inplace=True)
Datesfull.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4194300 entries, 0 to 4194299
Data columns (total 18 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   RecordedAtTime             object 
 1   DirectionRef               float64
 2   PublishedLineName          object 
 3   OriginName                 object 
 4   OriginLat                  float64
 5   OriginLong                 float64
 6   DestinationName            object 
 7   DestinationLat             float64
 8   DestinationLong            float64
 9   VehicleRef                 object 
 10  VehicleLocation.Latitude   float64
 11  VehicleLocation.Longitude  float64
 12  NextStopPointName          object 
 13  ArrivalProximityText       object 
 14  DistanceFromStop           object 
 15  ExpectedArrivalTime        object 
 16  ScheduledArrivalTime       object 
 17  None                       object 
dtypes: float64(7), object(11)
memory usage: 576.0+ MB


In [ ]:
# Due to local system memory constraints, file is reduced by 50%
Dates = Datesfull.sample(frac=.50)
Dates.reset_index(drop=True, inplace=True)

In [ ]:
# deleting previous dataframe to clear memory space
del Datesfull

In [ ]:
# New dataframe consists of 2 million rows and 18 features
Dates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2097150 entries, 0 to 2097149
Data columns (total 18 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   RecordedAtTime             object 
 1   DirectionRef               float64
 2   PublishedLineName          object 
 3   OriginName                 object 
 4   OriginLat                  float64
 5   OriginLong                 float64
 6   DestinationName            object 
 7   DestinationLat             float64
 8   DestinationLong            float64
 9   VehicleRef                 object 
 10  VehicleLocation.Latitude   float64
 11  VehicleLocation.Longitude  float64
 12  NextStopPointName          object 
 13  ArrivalProximityText       object 
 14  DistanceFromStop           object 
 15  ExpectedArrivalTime        object 
 16  ScheduledArrivalTime       object 
 17  None                       object 
dtypes: float64(7), object(11)
memory usage: 288.0+ MB


In [ ]:
# Dropping coordinate fields due to them not being used
# Dropping proximaty alert text due to redundancy in other features
# Dropping None field as it provides no explanation of purpose
Dates.drop(['None','OriginLat','OriginLong','OriginName','DestinationLat','DestinationLong','VehicleLocation.Latitude','VehicleLocation.Longitude','ArrivalProximityText'], axis=1, inplace=True)

In [ ]:
# Convert all destination names into potentially useful features
# with the help of one-hot encoding
test = Dates['DestinationName'].value_counts()
Dates['DestinationName'] = np.where(Dates['DestinationName'].isin(test.index[test < 2000]), 'Other', Dates['DestinationName'])
dummy = pd.get_dummies(Dates['DestinationName'], drop_first=True)
Dates = pd.concat([Dates, dummy], axis=1)

In [ ]:
# Converting the Published Line name to a new feature that
# removes the number, giving us only the route acronym, then
# applying one-hot encoding. Lastly dropping the original feature column
Dates = Dates.join(pd.get_dummies(Dates['PublishedLineName'].str.split('9|8|7|6|5|4|3|2|1|0').str[0],drop_first=True))

Dates.drop(['PublishedLineName'],axis=1, inplace=True)

In [ ]:
# During previous reprocessing, NaN values were somehow created, those are being dropped.
# Replacing Any value in the column Distance from stop that is considered
# "very close to arrival" to an interger of zero. Also converting to a smaller integer format.
Dates.dropna(inplace=True)
Dates['DistanceFromStop'].replace('at stop|approaching|< 1 stop away','0 ',regex=True, inplace=True)
Dates['DistanceFromStop'] = Dates['DistanceFromStop'].astype(int)

#Dates = Dates.dropna(axis=0, subset=['DirectionRef'])

In [ ]:
# Defining functions whose purpose is to reprocess the date time stamps
# located under Recorded Time, Scheduled Time, and Expected Time.
# Lastly dropping the original column names. 
def convertRecTime(x):
  Dates['RecTimeParsed'] = pd.to_datetime(x, format='%m/%d/%Y %H:%M')
  Dates['RecTime'] = Dates['RecTimeParsed'].dt.time
  return 

def convertSchedTime(x):
  Dates['SchedTimeParsed'] = pd.to_datetime(Dates['ScheduledArrivalTime'], format='%H:%M:%S', errors='coerce')
  Dates['SchedTime'] = Dates['SchedTimeParsed'].dt.time
  return 

def convertExpecTime(x):
  Dates['ExpecTimeParsed'] = pd.to_datetime(Dates['ExpectedArrivalTime'], format='%m/%d/%Y %H:%M', errors='coerce')
  Dates['ExpecTime'] = Dates['ExpecTimeParsed'].dt.time
  return

convertRecTime(Dates['RecordedAtTime'])
convertSchedTime(Dates['ScheduledArrivalTime'])
convertExpecTime(Dates['ExpectedArrivalTime'])

Dates.drop(['ExpectedArrivalTime','ScheduledArrivalTime','RecordedAtTime','RecTime','SchedTime','ExpecTime','DestinationName','NextStopPointName','VehicleRef'],axis=1, inplace=True)


In [ ]:
# Converting the newly created time for recorded date into
# the specific day and hour for potential use later. Also
# applying one-hot encoding to newly created features.

Dates['Day'] = Dates['RecTimeParsed'].dt.day_name()
Dates['Hour'] = Dates['RecTimeParsed'].dt.hour
Dates = Dates.join(pd.get_dummies(Dates['Day'],drop_first=True))

In [ ]:
# Dropping feature labeled recorded time
Dates.drop(['RecTimeParsed'],axis=1, inplace=True)

In [ ]:
# During previous analysis, serious issue was detected
# with overlaps of time crossing over from one day
# to the other. due to this, I am removing all arrival times
# within 5 hours of midnight.
# Lastly, defining "late" and "Early" which will be used to identify
# later in model buidling
indexName = Dates[Dates['Hour'] > 22].index
Dates.drop(indexName, inplace=True)
indexName2 = Dates[Dates['Hour'] < 3].index
Dates.drop(indexName2, inplace=True)

Dates['SecondsLate'] = (Dates['ExpecTimeParsed'].dt.second + Dates['ExpecTimeParsed'].dt.minute * 60 + Dates['ExpecTimeParsed'].dt.hour * 3600) - (Dates['SchedTimeParsed'].dt.second + Dates['SchedTimeParsed'].dt.minute * 60 + Dates['SchedTimeParsed'].dt.hour * 3600)

Dates['Late_yes'] = [1 if seconds > 900 else 0 for seconds in Dates['SecondsLate']]
Dates['Early_yes'] = [1 if seconds < 0 else 0 for seconds in Dates['SecondsLate']]


In [ ]:
# dropping all redudent fields
Dates.drop(['SchedTimeParsed','ExpecTimeParsed','Day','SecondsLate'],axis=1, inplace=True)

In [ ]:
# Reseting index once more and reducing all numbers remaining
# to smallest posible value to save memory space. This is
# necessary due to the amount of features that have been created
# during feature engineering
Dates.reset_index(drop=True, inplace=True)
Dates = Dates.astype('int8')

#**Merge Data**

In [ ]:
Dates.info()
Dates.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1693248 entries, 0 to 1693247
Columns: 367 entries, DirectionRef to Early_yes
dtypes: int8(367)
memory usage: 592.6 MB


,DirectionRef,DistanceFromStop,149 ST,227 ST 114 AV via LIBERTY,25 AV CROPSEY AV,31 ST 6 AV,41 ST via BROADWAY/7 AV,42 ST PIER CROSSTOWN,44 ST 6 AV,ABINGDON SQ CROSSTOWN,...,X,Hour,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Late_yes,Early_yes
0,0,-100,0,0,0,0,0,0,0,0,...,0,15,0,0,1,0,0,0,0,0
1,1,-49,0,0,0,0,0,0,0,0,...,1,18,0,0,0,1,0,0,0,0
2,0,87,0,0,0,0,0,0,0,0,...,0,8,0,0,0,1,0,0,0,0
3,1,11,0,0,0,0,0,0,0,0,...,0,9,0,0,0,0,0,0,0,0
4,0,2,0,0,0,0,0,0,0,0,...,0,7,0,0,0,0,1,0,0,0


In [ ]:
# Data is exported to CSV 
# which can now be loaded to other notebooks for model building
# in order to save memory and reprocessing time during
# experimentation
Dates.to_csv(r'\Final Supervised Model',index=False)